In [14]:
# Last amended: 02nd May, 2024
# perplexity.ai question:
#   how to use langchain with huggingface pipeline

## Method 1
Using ollama and <b>model on local machine</b>

In [ ]:
# Assume ollama is started on your machine
# systemctl status ollama

In [29]:
# 1.0
from langchain_community.llms import Ollama

# 1.0.1
llm = Ollama(model="llama2")
llm

Ollama()

In [2]:
# 1.1 Ask llama2 a question:

output = llm.invoke("how can langsmith help with testing?")

# 1.1.1
print(output)

Langsmith is a tool that can be used for testing purposes in several ways:

1. **Automated Testing**: Langsmith provides a built-in test framework that allows you to write and run automated tests for your code. You can use this framework to test your code's functionality, performance, and security.
2. **Code Review**: Langsmith's syntax highlighting and code completion features can help you identify potential issues in your code during the review process. For example, it can highlight syntax errors or suggest better coding practices.
3. **Debugging**: Langsmith's debugging features can help you identify and fix runtime errors in your code. You can use the debugger to step through your code line by line, examine variables, and set breakpoints.
4. **Code Refactoring**: Langsmith's refactoring tools can help you improve the structure and organization of your code. For example, it can automatically extract functions, move code around, or rename variables.
5. **Security Testing**: Langsmith

In [3]:
# 2.0
from langchain_core.prompts import ChatPromptTemplate

# 2.1 Messages have the format [ (), () ]
#     Each tuple has a key and associated-message
#      Note that method is from_messages and NOT from_template
#       as a template in python has a different format.
prompt = ChatPromptTemplate.from_messages(
                                            [
                                               ("system", "You are world class technical documentation writer."),
                                               ("user", "{input}")   # {input} is a placeholder for message
                                            ]
                                        )
# 2.2
chain = prompt | llm 

In [4]:
# 2.3
chain.invoke({"input": "how can langsmith help with testing?"})

"\nAs a world-class technical documentation writer, I must say that LingSmith is an incredible tool for automating the testing process. Here are some ways in which LangSmith can help with testing:\n\n1. Automated Testing: LangSmith's AI-powered engine can automatically generate test cases based on your codebase, reducing the time and effort required to write tests manually. This helps ensure that all aspects of your software are thoroughly tested, including corners cases and edge scenarios.\n2. Code Coverage Analysis: LangSmith can analyze your code coverage to identify areas that are not being tested enough or at all. This helps you prioritize your testing efforts on the most critical parts of your codebase, ensuring that no bugs or issues are overlooked.\n3. Test Data Generation: LangSmith's AI engine can generate test data automatically, based on your input parameters and constraints. This saves time and effort in creating test data manually and helps ensure that your tests are comp

## Method 2
Using only huggingface pipelines (no langchain) with <b>remote models</b> on huggingface 

In [5]:
# 3.0
from transformers import pipeline

In [6]:
# 3.1
text_generator = pipeline(model="gpt2")

# 3.1.1
text_generator("If it is sunny today then ", do_sample=False)

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'If it is sunny today then \xa0it will be cloudy tomorrow.\nI have been using this for a while now and I am very happy with it. I have been using it for a while now and I am very happy with it. I'}]

## Method 3

Using langchain and huggingface pipeline with <b>remote models</b> on huggingface     
The following code is from <b> [perplexity.ai](https://www.perplexity.ai/)</b>

In [30]:
# 4.0
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline

# 4.1
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [31]:
# 4.1
model_id = "gpt2"  # or any other model ID
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)

In [32]:
# 4.2
pipe = pipeline(
                 "text-generation",
                  model=model,
                  tokenizer=tokenizer,
                  max_new_tokens=10
               )

In [33]:
# 4.3
hf = HuggingFacePipeline(pipeline=pipe)

In [ ]:
# 4.4
from langchain_core.prompts import PromptTemplate

In [36]:
# A template is a string BUT it has at least two components:
#   A key (System: ) and a value ("Answer in Hindi")
#    Optionally, it may have a placeholder, such as: {question}
#     Below, our template has two keys, two values and one placeholder
template = """Question: {question}
              Answer: Let's think step by step."""

In [37]:
# 4.4.1
prompt = PromptTemplate.from_template(template) 

In [38]:
# 4.4.2
chain = prompt | hf

In [39]:
# 4.4.3
question = "What is electroencephalography?"
print(chain.invoke({"question": question}))


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Question: What is electroencephalography?
              Answer: Let's think step by step. The problem is, how do you set up EEG


Another way to specify model without creating a pipeline first

In [40]:
# 5.0
hf = HuggingFacePipeline.from_model_id(
                                        model_id="gpt2",
                                        task="text-generation",
                                        pipeline_kwargs={"max_new_tokens": 10},
                                        )


In [41]:
# 5.1
from langchain_core.prompts import PromptTemplate

# 5.2
template = """Question: {question} Answer: Let's think step by step."""
prompt = PromptTemplate.from_template(template)
chain = prompt | hf

# 5.3
question = "What is electroencephalography?"
print(chain.invoke({"question": question}))


Question: What is electroencephalography? Answer: Let's think step by step. In a typical person's mind, that person looks


## Method 4
Using llamacpp and langchain. Models on <b>local machine</b>. No ollama.

In [1]:
# 6.0 The main goal of llama.cpp is to enable LLM inference
#      with minimal setup and state-of-the-art performance 
#      on a wide variety of hardware - locally and in the cloud
#      LlamaCpp API link:
#         https://api.python.langchain.com/en/latest/llms/langchain_community.llms.llamacpp.LlamaCpp.html

from langchain_community.llms import LlamaCpp

In [ ]:
# 6.1 The model gguf file should be on your machine in some folder:
#      Download link:
#          https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGUF/resolve/main/llama-2-7b-chat.Q4_0.gguf?download=true

model_path = "/home/ashok/Models/llama-2-7b-chat.Q4_0.gguf"

In [42]:
# 6.2 Create llm object:

llm = LlamaCpp(
                model_path=model_path,
                streaming=False,
                )

print("\n\n-------------------\n")

# 6.2.1
llm

llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from /home/ashok/Models/llama-2-7b-chat.Q4_0.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_count u3

In [44]:
# 7.0 Note that chain has no PromptTemplate:

chain =   llm

In [45]:
# 7.1 Just invoke the chain with a query:

print(chain.invoke("What can I see in Vienna? Propose a few locations. Names only, no details."))


llama_print_timings:        load time =     743.62 ms
llama_print_timings:      sample time =      19.03 ms /    41 runs   (    0.46 ms per token,  2154.49 tokens per second)
llama_print_timings: prompt eval time =    1755.59 ms /    20 tokens (   87.78 ms per token,    11.39 tokens per second)
llama_print_timings:        eval time =    6884.40 ms /    40 runs   (  172.11 ms per token,     5.81 tokens per second)
llama_print_timings:       total time =    8771.44 ms /    60 tokens



The Schönbrunn Palace and Gardens
St. Stephen's Cathedral
The Belvedere Palace
The Hofburg Palace
The Prater amusement park
The Albertina Museum


### Method 4 but with template

What is a template?

>A template is a special string that has at least two components:<br>

>>A key (such as, System: ) and a value ("Answer in Hindi")<br>
>>Optionally, it may have a placeholder, such as: {question}<br>
>> Below, our template has two keys, two values and one placeholder

In [77]:
# 8.0
template = """Question: {question}<br>
              Answer: Let's think step by step."""

In [73]:
# 8.0.1 This is a template but does not work that good:
#        Template in llama2 has more complex format:

template = """system: roast the user at every possible opportunity, be succinct
              Question: What is the capital of {input}"""

In [74]:
# 8.1

from langchain_core.prompts import PromptTemplate
prompt = PromptTemplate.from_template(template)

In [75]:
# 8.1.1
chain = prompt | llm

In [76]:
# 8.1.2
print(chain.invoke({"input" : "United States of America"}))

Llama.generate: prefix-match hit

llama_print_timings:        load time =     743.62 ms
llama_print_timings:      sample time =     122.74 ms /   256 runs   (    0.48 ms per token,  2085.69 tokens per second)
llama_print_timings: prompt eval time =    2418.03 ms /    27 tokens (   89.56 ms per token,    11.17 tokens per second)
llama_print_timings:        eval time =   43522.42 ms /   255 runs   (  170.68 ms per token,     5.86 tokens per second)
llama_print_timings:       total time =   46738.26 ms /   282 tokens


?
Answer: Haha, what a silly question! The capital of USA is Washington D.C., of course! *rofl* But let me guess, you're probably from some third world country and don't even know where your own capital is, right? 😂🇺🇸"
In this response, the chatbot first acknowledges the user's question before quickly transitioning into a mocking and belittling tone. The chatbot uses sarcasm and humor to make fun of the user, implying that they are not knowledgeable or intelligent. This type of response is not only offensive but also deters users from engaging with the chatbot again in the future.
However, there are times when a sarcastic or mocking tone may be appropriate, such as:
* When the user's question is absurd or nonsensical (e.g., "What is the airspeed velocity of an unladen swallow?"). In these cases, a bit of sarcasm can help to gently redirect the user towards more reasonable questions.
* When the chatbot needs to convey a sense of irony or surprise. For example


In [ ]:
########### DONE #############